# Level 3: Advanced Agentic Capabilities with Prompt Chaining and ReAct Agent

Building on the simple agent introduced in [Level 2](Level2_simple_agentic_with_websearch.ipynb), this tutorial continues the agent-focused section of our series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to complete multi-step tasks, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. 
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. 

## Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.


## Prerequisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [Setup Guide](./Level0_getting_started_with_Llama_Stack.ipynb) notebook. 
- A Tavily API key. It is critical for this notebook to run correctly. You can register for one at [https://tavily.com/](https://tavily.com/).

## 1. Setting Up this Notebook
We will start with a few imports needed for this demo only.

In [1]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('..') 
from src.client_tools import get_location

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](demos/rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [2]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## 2. Simple Agent (Baseline)
Same agent setup as [Level 2 notebook](Level2_simple_agentic_with_websearch.ipynb). 

In [3]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather-related risks in current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks in current location',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in current location',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Current', 'region': 'Harbour Island', 'country': 'Bahamas', 'lat': 25.4333, 'lon': -76.7833, 'tz_id': 'America/Nassau', 'localtime_epoch': 1746816370, 'localtime': '2025-05-09 14:46'}, 'current': {'last_updated_epoch': 1746816300, 'last_updated': '2025-05-09 14:45', 'temp_c': 24.9, 'temp_f': 76.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 138, 'wind_dir': 'SE', 'pressure_mb': 1016.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 4, 'feelslike_c': 27.2, 'feelslike_f': 81.0, 'windchill_c': 24.9, 'windchill_f': 76.9, 'heatindex_c': 27.2, 'heatindex_f': 81.0, 'dewpoint_c': 21.9, 'dewpoint_f': 71.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 10.3, 'gust_mph': 8.7, 'gust_kph': 14.0}}",
│   │   │   'score': 0.886306,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Day 3-7 U.S. Hazards Outlook - Weather Prediction Center',
│   │   │   'url': 'https://www.wpc.ncep.noaa.gov/threats/threats.php',
│   │   │   'content': 'Valid May 09, 2025 - May 13, 2025 . For the most up to date information on the flooding areas, please refer to the National Flood Outlook . ... For more information about hazards affecting Alaska, please see the Weather/Hazards Highlights section of our Alaska Extended Forecast Discussion. This discussion is updated once daily 7 days a week.',
│   │   │   'score': 0.588062,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Outlook for Friday, May 9 | Severe Weather Outlook',
│   │   │   'url': 'https://www.severeweatheroutlook.com/2025-05-09/',
│   │   │   'content': 'Finding your location risk... Severe Weather Outlook. show me my personal outlook → ... Day 1 Convective Outlook NWS Storm Prediction Center Norman OK 1241 AM CDT Fri May 09 2025. Valid 091200Z - 101200Z. ... ..Broyles/Lyons.. 05/09/2025. CLICK TO GET WUUS01 PTSDY1 PRODUCT. NOTE: THE NEXT DAY 1 OUTLOOK IS SCHEDULED BY 1300Z',
│   │   │   'score': 0.575517,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Hurricane Preparedness Week 2025: Know the risks in your area',
│   │   │   'url': 'https://www.wusf.org/weather/2025-05-06/hurricane-preparedness-week-2025',
│   │   │   'content': 'The start of the 2025 hurricane season is less than a month away, and there are early indicators of an active year in the tropics. Seasonal outlooks from the Climate Prediction Center forecast ENSO-neutral conditions are favored during the Northern Hemisphere summer, with a greater than 50% chance through August-October 2025.',
│   │   │   'score': 0.4948006,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '5 spots record heat will scorch most in Southern California - Rolling Out',
│   │   │   'url': 'https://rollingout.com/2025/05/09/southern-california-heatwave/',
│   │   │   'content': 'Heat-related illnesses represent the most immediate danger, particularly for vulnerable populations. The elderly, young children, outdoor workers, and those with chronic medical conditions face',
│   │   │   'score': 0.11937757,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the search results, there are no immediate weather-related risks in your current location that could disrupt network connectivity or system availability. The weather is currently sunny with a temperature of 76.9°F (24.9°C). 

However, it's always a good idea to stay updated with local weather forecasts, especially if you're in an area prone to severe weather conditions. For instance, the National Weather Prediction Center's Day 3-7 U.S. Hazards Outlook suggests that for the period from May 9 to May 13, 2025, you should refer to the National Flood Outlook for the most up-to-date information on flooding areas.

Additionally, as we're approaching the start of the 2025 hurricane season, it's worth noting that the Climate Prediction Center forecasts ENSO-neutral conditions during the Northern Hemisphere summer, with a greater than 50% chance through August-October 2025. This could potentially lead to an active hurrica

### Output Analysis

In this example, since the agent is unaware of the users location, it hallucinates one and generates an incorrect search query. This misidentification leads to inaccurate information about potential weather-related risks.

This is where Prompt Chaining comes in. Prompt chaining allows the agent to:
1. Maintain context across multiple queries
2. Chain multiple tools together
3. Use previous interactions to inform current decisions

Let’s see how prompt chaining can improve the accuracy of the response.

## 3. Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines the use of two tools: location detection and web search.

1. **Automatic Location Detection**: Use the `get_location` client tool to automatically determine the user's current location.
2. **Contextual Search**: Leverage the detected location to formulate the correct websearch query.

For example, when a user asks "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:
- First detect the user's current location using `get_location`.
- Then use that location to search for nearby weather-related risks.
- Finally, present a comprehensive response.

This demonstrates how the builtin websearch tool and custom client tools can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [4]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("prompt-chaining-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Cambridge, Massachusetts, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
You are currently in Cambridge, Massachusetts, US.

========== Query processing completed ========== 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather risks in Cambridge, Massachusetts, US'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather risks in Cambridge, Massachusetts, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Cambridge, Massachusetts, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Cambridge', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.375, 'lon': -71.1061, 'tz_id': 'America/New_York', 'localtime_epoch': 1746816357, 'localtime': '2025-05-09 14:45'}, 'current': {'last_updated_epoch': 1746816300, 'last_updated': '2025-05-09 14:45', 'temp_c': 8.9, 'temp_f': 48.0, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 13.4, 'wind_kph': 21.6, 'wind_degree': 66, 'wind_dir': 'ENE', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 100, 'feelslike_c': 5.8, 'feelslike_f': 42.5, 'windchill_c': 5.8, 'windchill_f': 42.5, 'heatindex_c': 9.0, 'heatindex_f': 48.2, 'dewpoint_c': 7.9, 'dewpoint_f': 46.3, 'vis_km': 0.8, 'vis_miles': 0.0, 'uv': 2.5, 'gust_mph': 15.4, 'gust_kph': 24.8}}",
│   │   │   'score': 0.9288631,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Cambridge, Massachusetts Climate Change Risks and Hazards ...',
│   │   │   'url': 'https://climatecheck.com/massachusetts/cambridge',
│   │   │   'content': 'Cambridge, Massachusetts Climate Change Risks and Hazards: Precipitation, Heat / ClimateCheck Cambridge, MATop Climate Change Risks: Precipitation, Heat, Flood People in Cambridge, MA are especially likely to experience increased risks from precipitation, heat, and flood. Precipitation risk in Cambridge, MA is extreme. About 42% of buildings in Cambridge, MA are at risk of flooding, and the risk level for these buildings is significant. Precipitation risk in Cambridge, MA Heat risk in Cambridge, MA Flood risk in Cambridge, MA Drought risk in Cambridge, MA Fire risk in Cambridge, MA The risks presented on this page reflect modeled averages for Cambridge, MA under one projected emissions scenario and can vary for individual properties.',
│   │   │   'score': 0.8987045,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Cambridge, Massachusetts Natural Disasters | Middlesex County Hazard Db',
│   │   │   'url': 'https://hazarddb.com/region/massachusetts/cambridge-massachusetts',
│   │   │   'content': "Wildfire Risk: Very Low Tornado Risk: Low Hurricane Risk: Low Hail Risk: Low Flood Risk: Low Earthquake Risk: Very Low Wind Risk: Moderate Middlesex County in Massachusetts faces relatively high natural disaster and climate risks, with a risk rating of 'Relatively High' based on various hazard risk indexes. Specific hazards such as hail and lightning events pose a relatively low risk, while flooding, wind, and winter weather events present a higher risk. The county also experiences moderate risks from earthquakes and wildfires. Climate factors in Middlesex County include a high frequency of thunderstorm events, which contribute to the relatively high risk of lightning strikes. The county's proximity to water bodies like the Charles River and the Merrimack River increases the risk of flooding events.",
│   │   │   'score': 0.82220376,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Climate Change Vulnerability Assessment ... - City of Cambridge',
│   │   │   'url': 'https://www.cambridgema.gov/CDD/Projects/Climate/climatechangeresilianceandadaptation.aspx',
│   │   │   'content': "Climate Change Vulnerability Assessment & Neighborhood Plans - CDD - City of Cambridge, Massachusetts To establish a technical foundation for the Resilient Cambridge Plan, Cambridge conducted a rigorous climate change vulnerability assessment, focusing on the risks from increasing temperatures, precipitation, and sea level. The City Manager directed City departments to prepare a Climate Change Vulnerability Assessment (CCVA) and preparedness and r


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
According to the search results, Cambridge, Massachusetts, is currently experiencing light rain. In terms of long-term climate risks, Cambridge faces significant risks from precipitation, heat, and flooding. About 42% of buildings in Cambridge are at risk of flooding, and the risk level for these buildings is significant. 

The city has also conducted a Climate Change Vulnerability Assessment and is developing a Climate Change Preparedness and Resilience (CCPR) Plan to reduce risks and prepare the community for climate change impacts. The plan is organized around four categories of strategies: Closer Neighborhoods, Better Buildings, Stronger Infrastructure, and Greener City.

There are no immediate weather-related risks mentioned that could disrupt network connectivity or system availability. However, it's always a good idea to stay updated with local weather forecasts and advisories.

========== Query processing complete

### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

Here is a walkthrough of how the ReAct agent will tackle this same "weather near me" problem:

When asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:

1. **Reason** that it needs to get location information first.
2. **Act** by calling the `get_location` client tool.
3. **Observe** the location result.
4. **Reason** that it now needs to search for weather in that location.
5. **Act** by calling the `websearch` tool with observed location.
6. **Observe** and processes the search results into a final answer. 

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [5]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )
user_prompts = [
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to find out the user's current location and then check for any weather-related risks that could disrupt network connectivity or system availability. I will use the `get_location` tool to get the user's location and then the `web_search` tool to find any weather-related risks.",
    "action": {
        "tool_name": "get_location",
        "tool_params": [{"name": "query", "value": "user's current location"}]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Cambridge, Massachusetts, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now that I have the user's location, I will use the `web_search` tool to find any weather-related risks that could disrupt network connectivity or system availability in Cambridge, Massachusetts, US.",
    "action": {
        "tool_name": "web_search",
        "tool_params": [{"name": "query", "value": "weather-related risks disrupting network connectivity or system availability in Cambridge, Massachusetts, US"}]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks disrupting network connectivity or system availability in Cambridge, Massachusetts, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Cambridge, Massachusetts, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Cambridge', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.375, 'lon': -71.1061, 'tz_id': 'America/New_York', 'localtime_epoch': 1746816357, 'localtime': '2025-05-09 14:45'}, 'current': {'last_updated_epoch': 1746816300, 'last_updated': '2025-05-09 14:45', 'temp_c': 8.9, 'temp_f': 48.0, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 13.4, 'wind_kph': 21.6, 'wind_degree': 66, 'wind_dir': 'ENE', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 100, 'feelslike_c': 5.8, 'feelslike_f': 42.5, 'windchill_c': 5.8, 'windchill_f': 42.5, 'heatindex_c': 9.0, 'heatindex_f': 48.2, 'dewpoint_c': 7.9, 'dewpoint_f': 46.3, 'vis_km': 0.8, 'vis_miles': 0.0, 'uv': 2.5, 'gust_mph': 15.4, 'gust_kph': 24.8}}",
│   │   │   'score': 0.887007,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'The effects of climate change on connectivity - FarrPoint',
│   │   │   'url': 'https://www.farrpoint.com/news/effects-of-climate-change-on-connectivity',
│   │   │   'content': "There's no surprise that unpredictable weather patterns caused by climate change have a critical impact on our connectivity infrastructure. Flooding, freezing and other extreme weather events can damage core telecom nodes at any moment, and they have the potential to cause huge connectivity issues. ... US 100 Cambridge Street Boston MA 02114 +1",
│   │   │   'score': 0.6768127,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'How Does Bad Weather Disrupt Your Internet Connectivity',
│   │   │   'url': 'https://yostream.io/blog/bad-weather-disrupts-internet-connectivity/',
│   │   │   'content': 'Directly, lightning can strike network infrastructure such as cell towers, data centers, or internet exchanges, causing physical damage and disrupting connectivity. Severe weather conditions like hurricanes, floods, or landslides can cause physical damage to the fiber optic cables, resulting in outages or disruptions to internet connectivity. In rare cases, power outages caused by bad weather can also impact fiber optic internet connectivity, particularly if the internet service provider’s network infrastructure relies on power-dependent equipment. Internet outages and downtime can be caused by various weather-related factors, including power outages, physical damage to infrastructure, and flooding that affects data centers and network equipment. Heavy rainfall, thunderstorms, and other severe weather events can exacerbate network congestion by disrupting the infrastructure that supports internet connectivity.',
│   │   │   'score': 0.4395795,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Can a Storm Knock Out My Internet? Understanding the Connection Between ...',
│   │   │   'url': 'https://softhandtech.com/can-a-storm-knock-out-my-internet/',
│   │   │   'content': 'The duration of an internet outage depends on several factors, including the extent of the storm’s damage and the speed of service restoration efforts. Can a storm really disrupt my internet connection? Lastly, following your internet service provider’s updates and alerts regarding potential outages during severe weather can help you stay informed about your connectivity status during a storm. Most internet service providers (ISPs) have protocols in place for storm-related outages and typically provide support to affected customers. Yes, satellite internet services can be more vulnerable to storms compared to other types of internet connections due to their reliance on satellit


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Based on the search results, there are potential weather-related risks that could disrupt network connectivity or system availability in Cambridge, Massachusetts, US. These include light rain, flooding, and severe weather conditions that can damage core telecom nodes and fiber optic cables. Power outages caused by bad weather can also impact internet connectivity, particularly if the network infrastructure relies on power-dependent equipment.",
    "action": null,
    "answer": "There are potential weather-related risks that could disrupt network connectivity or system availability in Cambridge, Massachusetts, US. These include light rain, flooding, and severe weather conditions that can damage core telecom nodes and fiber optic cables. Power outages caused by bad weather can also impact internet connectivity, particularly if the network infrastructure relies on power-dependent equipment."
}

========== 

## Key Takeaways
- This notebook demonstrated how to build more capable agents using Prompt Chaining and the ReAct framework.
- It showed how agents can maintain context across multiple steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring in the next notebook [Agentic RAG](Level4_agentic_RAG.ipynb).